In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from matplotlib import pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve

import pickle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import os

import string

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_addons as tfa

In [2]:
df = pd.read_table('/kaggle/input/fb-post-data/FB_posts_labeled.txt')

In [3]:
df['text'] = df['message'].map(lambda text: text.translate(str.maketrans('','',string.punctuation)).strip())
df['text'] = df['text'].map(lambda text: text.translate(str.maketrans('','',string.digits)).strip())

In [4]:
##Only use while reading 1 dataset
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df[['Appreciation', 'Complaint', 'Feedback']], 
                                                                    random_state=69, 
                                                                    test_size=0.3, 
                                                                    stratify=df[['Appreciation', 'Complaint', 'Feedback']])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=69, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [5]:
print(train_labels.sum(axis = 0) / train_labels.shape[0])
print(val_labels.sum(axis = 0) / val_labels.shape[0])
print(test_labels.sum(axis = 0) / test_labels.shape[0])

Appreciation    0.258973
Complaint       0.534458
Feedback        0.206569
dtype: float64
Appreciation    0.258794
Complaint       0.534338
Feedback        0.206868
dtype: float64
Appreciation    0.259414
Complaint       0.534728
Feedback        0.205858
dtype: float64


In [6]:
# X_train = df['text'].values
# y_train = np.array(df[['Appreciation', 'Complaint', 'Feedback']])

In [7]:
# print(X_train.shape)
# print(y_train.shape)

### Model building

In [8]:
#encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
#encoder = hub.load('https://tfhub.dev/google/universal-sentence-encoder-large/5')

In [27]:
# import keras.backend as K

# def get_f1(y_true, y_pred): #taken from old keras source code
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     recall = true_positives / (possible_positives + K.epsilon())
#     f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
#     return f1_val

In [33]:
f1 = tfa.metrics.F1Score(num_classes = 3, average = 'macro')

callbacks = [
    tf.keras.callbacks.EarlyStopping(     #EarlyStopping is used to stop at the epoch where val_accuracy does not improve significantly
        monitor='val_loss',
        min_delta=1e-4,
        patience=5,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='weights_use.h5',
        monitor='val_loss', 
        mode='min', 
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

In [34]:
def USE_model_builder():
    model = tf.keras.models.Sequential()
    model.add(hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder-large/5', 
                        input_shape=[], 
                        dtype=tf.string, 
                        trainable=False))
    model.add(tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.01)))
    #model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics= [f1])
    
    return model

In [35]:
use_model = USE_model_builder()
use_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 512)               147354880 
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 3)                 771       
                                                                 
Total params: 147,486,979
Trainable params: 132,099
Non-trainable params: 147,354,880
_________________________________________________________________


In [ ]:
use_model.fit(train_text, 
          train_labels, 
          epochs=30, 
          callbacks=callbacks,
          batch_size = 8,
          validation_data= (val_text, val_labels)
          ,class_weight ={0: 1, 1:1, 2: 5}
          )

In [13]:
use_model.evaluate(test_text, test_labels)

38/38 [==============================] - 3s 82ms/step - loss: 0.5782 - f1_score: 0.8302


[0.5781764984130859, 0.8302238583564758]

In [14]:
raw_preds = use_model.predict(test_text)

38/38 [==============================] - 8s 72ms/step


In [15]:
raw_preds

array([[0.79829335, 0.15118171, 0.05052496],
       [0.00245451, 0.00559484, 0.99195063],
       [0.17105225, 0.80803543, 0.02091229],
       ...,
       [0.17585634, 0.8026761 , 0.02146755],
       [0.05746435, 0.02150149, 0.92103416],
       [0.04490947, 0.86052513, 0.09456541]], dtype=float32)

In [17]:
from numpy import argmax
for record in raw_preds:
    max_index =argmax(record)
    for i in range(3):
        if i == max_index:
            record[i] = 1
        else:
            record[i] = 0

In [20]:
print(raw_preds)
print(np.array(test_labels))

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
[[1 0 0]
 [0 0 1]
 [0 1 0]
 ...
 [1 0 0]
 [0 0 1]
 [0 1 0]]


In [21]:
print(classification_report(np.array(test_labels), raw_preds))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       310
           1       0.92      0.77      0.84       639
           2       0.66      0.92      0.77       246

   micro avg       0.83      0.83      0.83      1195
   macro avg       0.82      0.86      0.83      1195
weighted avg       0.86      0.83      0.84      1195
 samples avg       0.83      0.83      0.83      1195



### HP tuning

In [6]:
pip install -q -U keras-tuner

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.82.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [36]:
import keras_tuner as kt

In [37]:
f1 = tfa.metrics.F1Score(num_classes = 3, average = 'macro')

In [40]:
def USE_model_builder_HP(hp):
    model = tf.keras.models.Sequential()
    model.add(hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder-large/5', 
                        input_shape=[], 
                        dtype=tf.string, 
                        trainable=False))
    
    #Layer 1
    hp_units = hp.Int('units', min_value=192, max_value=512, step=32)
    hp_l2 = hp.Choice('l2', values=[1e-1, 1e-2, 1e-3, 1e-4])
    model.add(tf.keras.layers.Dense(units = hp_units, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(hp_l2)))
    
    #Layer 2
    model.add(tf.keras.layers.Dense(units = hp_units, activation='relu'))
    
    #Layer 3
    hp_dropout = hp.Choice('Dropout', values=[0.0, 0.05,0.1])
    model.add(tf.keras.layers.Dropout(hp_dropout))
    
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) 
    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='categorical_crossentropy',
                metrics=[f1])
    
    return model

In [41]:
tuner = kt.Hyperband(USE_model_builder_HP,
                     objective= kt.Objective("val_f1_score", direction="max"),
                     max_epochs=10,
                     factor=4)

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score', patience=5)

tuner.search(train_text, train_labels,
             epochs= 5,
             validation_data=(val_text, val_labels), 
             callbacks=[stop_early],
            class_weight ={0: 1, 1:1, 2: 5})

Trial 11 Complete [00h 04m 49s]
val_f1_score: 0.8445949554443359

Best val_f1_score So Far: 0.8590073585510254
Total elapsed time: 00h 33m 51s


In [42]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

USE_tuned_model = tuner.hypermodel.build(best_hps)
history = USE_tuned_model.fit(train_text, train_labels, epochs=30, validation_data=(val_text, val_labels))

val_f1_per_epoch = history.history['val_f1_score']
best_epoch = val_f1_per_epoch.index(max(val_f1_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/30
175/175 [==============================] - 56s 155ms/step - loss: 0.4870 - f1_score: 0.8033 - val_loss: 0.3976 - val_f1_score: 0.8478
Epoch 2/30
175/175 [==============================] - 19s 107ms/step - loss: 0.3204 - f1_score: 0.8855 - val_loss: 0.4072 - val_f1_score: 0.8487
Epoch 3/30
175/175 [==============================] - 21s 123ms/step - loss: 0.2649 - f1_score: 0.9102 - val_loss: 0.4102 - val_f1_score: 0.8568
Epoch 4/30
175/175 [==============================] - 21s 121ms/step - loss: 0.2020 - f1_score: 0.9414 - val_loss: 0.4504 - val_f1_score: 0.8504
Epoch 5/30
175/175 [==============================] - 18s 104ms/step - loss: 0.1396 - f1_score: 0.9730 - val_loss: 0.5078 - val_f1_score: 0.8474
Epoch 6/30
175/175 [==============================] - 21s 119ms/step - loss: 0.1028 - f1_score: 0.9850 - val_loss: 0.5954 - val_f1_score: 0.8512
Epoch 7/30
175/175 [==============================] - 18s 104ms/step - loss: 0.0824 - f1_score: 0.9909 - val_loss: 0.6366 - val_f1

In [43]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(train_text, train_labels, epochs=best_epoch, validation_data=(val_text, val_labels))

Epoch 1/3
175/175 [==============================] - 54s 150ms/step - loss: 0.4864 - f1_score: 0.8131 - val_loss: 0.3873 - val_f1_score: 0.8599
Epoch 2/3
175/175 [==============================] - 21s 119ms/step - loss: 0.3228 - f1_score: 0.8862 - val_loss: 0.3955 - val_f1_score: 0.8629
Epoch 3/3
175/175 [==============================] - 19s 110ms/step - loss: 0.2675 - f1_score: 0.9103 - val_loss: 0.4070 - val_f1_score: 0.8552


In [12]:
eval_result = hypermodel.evaluate(test_text, test_labels)
print("[test loss, test f1]:", eval_result)

38/38 [==============================] - 1s 16ms/step - loss: 0.5263 - f1_score: 0.8324
[test loss, test f1]: [0.5263124108314514, 0.8324432373046875]


In [44]:
raw_preds = hypermodel.predict(test_text)

from numpy import argmax
for record in raw_preds:
    max_index =argmax(record)
    for i in range(3):
        if i == max_index:
            record[i] = 1
        else:
            record[i] = 0
print(raw_preds)
print(np.array(test_labels))

38/38 [==============================] - 8s 77ms/step
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
[[1 0 0]
 [0 0 1]
 [0 1 0]
 ...
 [1 0 0]
 [0 0 1]
 [0 1 0]]


In [45]:
print(classification_report(np.array(test_labels), raw_preds))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       310
           1       0.91      0.87      0.89       639
           2       0.78      0.83      0.81       246

   micro avg       0.88      0.88      0.88      1195
   macro avg       0.86      0.88      0.87      1195
weighted avg       0.88      0.88      0.88      1195
 samples avg       0.88      0.88      0.88      1195



In [46]:
hypermodel.save('fb_hyper_use_large_class.hdf5')

In [47]:
hypermodel.save_weights('fb_hyper_use_large_class_weights.hdf5')

### Loading model

In [15]:
KerasLayer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder-large/5', 
                        input_shape=[], 
                        dtype=tf.string, 
                        trainable=False)

In [ ]:
loaded_model = tf.keras.models.load_model('/kaggle/input/use-hyper-tuned-model/USE_hyper.hdf5', custom_objects={'KerasLayer': KerasLayer})

### Adding predictions

In [17]:
predict_df = pd.read_table('/kaggle/input/fb-post-data/FB_posts_unlabeled.txt')

In [19]:
predict_df.head()

,postId,message
0,108381603303_10151119973393304,Love. It. To
1,115568331790246_371841206162956,NICE
2,115568331790246_515044031842672,Congrats
3,147285781446_10151010892176447,Awesome!
4,159616034235_10150639103634236,Award


In [20]:
predict_df['text'] = predict_df['message'].map(lambda text: text.translate(str.maketrans('','',string.punctuation)).strip())
predict_df['text'] = predict_df['text'].map(lambda text: text.translate(str.maketrans('','',string.digits)).strip())

In [21]:
raw_preds = hypermodel.predict(predict_df['text'])

64/64 [==============================] - 11s 75ms/step


In [24]:
from numpy import argmax

In [26]:
raw_preds

array([[9.9305809e-01, 6.6145551e-03, 3.2736102e-04],
       [9.9248600e-01, 6.7224773e-03, 7.9150259e-04],
       [9.9591041e-01, 3.6108098e-03, 4.7885551e-04],
       ...,
       [4.8022270e-01, 4.0724552e-01, 1.1253183e-01],
       [2.2773218e-01, 7.6742977e-01, 4.8380508e-03],
       [1.4850210e-01, 8.4430104e-01, 7.1968627e-03]], dtype=float32)

In [28]:
for record in raw_preds:
    max_index =argmax(record)
    for i in range(3):
        if i == max_index:
            record[i] = 1
        else:
            record[i] = 0

In [33]:
post_ids = np.array(predict_df['postId'])

In [41]:
raw_preds = raw_preds.astype('int')

In [47]:
# Don't run before adaptation, this is pseudocode!
f = open('predictions_v1.csv', 'w')
# write the first header row
f.write('postId,Appreciation_pred,Complaint_pred,Feedback_pred' + '\n')

for i in range(post_ids.size):
    Appreciation_pred, Complaint_pred, Feedback_pred = raw_preds[i][0], raw_preds[i][1], raw_preds[i][2]
    if Appreciation_pred not in [0,1] or Complaint_pred not in [0,1] or Feedback_pred not in [0,1]:
        print('did you forget to turn probability predictions into binary predictions?')
    if Appreciation_pred + Complaint_pred + Feedback_pred != 1:
        print('SOMETHING_IS_WRONG')
    f.write(post_ids[i] + ',' + str(Appreciation_pred) + ',' + str(Complaint_pred) + ',' + str(Feedback_pred) + '\n')
f.close()

### Loading model